In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import os
import time
import seaborn as sns
import lightgbm as lgb
import pickle
from datetime import date, timedelta
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

In [3]:
# 获取沪深300指数当前（）成分股
ts_api = ts.pro_api('f75ac7a71ba8bf7118966e7fb1a0224dda2345fa0f54d1f7b010b91b')
zz500 = ts_api.index_weight(index_code='000905.SH', start_date='20230201', end_date='20230228')
zz500

,index_code,con_code,trade_date,weight
0,000905.SH,600985.SH,20230228,0.209
1,000905.SH,600764.SH,20230228,0.083
2,000905.SH,600315.SH,20230228,0.160
3,000905.SH,600867.SH,20230228,0.232
4,000905.SH,002603.SZ,20230228,0.363
...,...,...,...,...
495,000905.SH,002673.SZ,20230228,0.258
496,000905.SH,600131.SH,20230228,0.117
497,000905.SH,600801.SH,20230228,0.172
498,000905.SH,002468.SZ,20230228,0.109


In [157]:
# 以000001.SZ为例搭建XGBoost预测模型
data = ts_api.daily(ts_code='000001.SZ')
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20230227,13.75,13.88,13.68,13.69,13.86,-0.17,-1.2266,621461.93,8.548540e+05
1,000001.SZ,20230224,14.02,14.03,13.80,13.86,14.05,-0.19,-1.3523,729989.04,1.012653e+06
2,000001.SZ,20230223,14.00,14.32,13.98,14.05,14.02,0.03,0.2140,824490.68,1.165540e+06
3,000001.SZ,20230222,14.00,14.12,13.94,14.02,14.10,-0.08,-0.5674,638922.39,8.957425e+05
4,000001.SZ,20230221,14.06,14.20,13.92,14.10,14.15,-0.05,-0.3534,990494.99,1.393153e+06
...,...,...,...,...,...,...,...,...,...,...,...
5995,000001.SZ,19970926,20.78,20.89,20.10,20.82,20.74,0.08,0.3900,59529.00,1.222137e+05
5996,000001.SZ,19970925,21.02,21.26,20.45,20.74,20.45,0.29,1.4200,103770.00,2.155430e+05
5997,000001.SZ,19970924,19.80,20.60,19.60,20.45,19.19,1.26,6.5700,154245.00,3.113654e+05
5998,000001.SZ,19970923,20.00,20.20,18.80,19.19,20.22,-1.03,-5.0900,118363.00,2.307110e+05


In [7]:
for code in tqdm(zz500['con_code']):
    data = ts_api.daily(ts_code=code, end_date = '20211231')
    if data.shape[0] > 1000:
        open_pct = (100*data['open'].loc[::-1].diff().loc[len(data)-2:].values/data['open'].loc[:1:-1].values).tolist()
        high_pct = (100*data['high'].loc[::-1].diff().loc[len(data)-2:].values/data['high'].loc[:1:-1].values).tolist()
        low_pct = (100*data['low'].loc[::-1].diff().loc[len(data)-2:].values/data['low'].loc[:1:-1].values).tolist()
        close_pct = (100*data['close'].loc[::-1].diff().loc[len(data)-2:].values/data['close'].loc[:1:-1].values).tolist()
        vol_pct = (100*data['vol'].loc[::-1].diff().loc[len(data)-2:].values/data['vol'].loc[:1:-1].values).tolist()
        amount_pct = (100*data['amount'].loc[::-1].diff().loc[len(data)-2:].values/data['amount'].loc[:1:-1].values).tolist()
        X = []
        y = []
        for i in range(len(open_pct)-5):
            X.append(open_pct[i:i+5] + high_pct[i:i+5] + low_pct[i:i+5] + close_pct[i:i+5] + vol_pct[i:i+5] + amount_pct[i:i+5])
            y.append(close_pct[i+5])
        model = lgb.LGBMRegressor(learning_rate=0.01, metric='l2')
        model.fit(X,y,eval_metric='l2')
        filename = 'zzmodel/{}.pickle'.format(code[:-3])
        pickle.dump(model, open(filename,'wb'))

100%|██████████| 500/500 [19:53<00:00,  2.39s/it]


In [216]:
# Unimodel
X = []
y = []
for code in tqdm(hs300['con_code']):
    data = ts_api.daily(ts_code=code)
    open_pct = (100*data['open'].loc[::-1].diff().loc[len(data)-2:].values/data['open'].loc[:1:-1].values).tolist()
    high_pct = (100*data['high'].loc[::-1].diff().loc[len(data)-2:].values/data['high'].loc[:1:-1].values).tolist()
    low_pct = (100*data['low'].loc[::-1].diff().loc[len(data)-2:].values/data['low'].loc[:1:-1].values).tolist()
    close_pct = (100*data['close'].loc[::-1].diff().loc[len(data)-2:].values/data['close'].loc[:1:-1].values).tolist()
    vol_pct = (100*data['vol'].loc[::-1].diff().loc[len(data)-2:].values/data['vol'].loc[:1:-1].values).tolist()
    amount_pct = (100*data['amount'].loc[::-1].diff().loc[len(data)-2:].values/data['amount'].loc[:1:-1].values).tolist()
    for i in range(len(open_pct)-5):
        X.append(open_pct[i:i+5] + high_pct[i:i+5] + low_pct[i:i+5] + close_pct[i:i+5] + vol_pct[i:i+5] + amount_pct[i:i+5])
        y.append(close_pct[i+5])
    
model = lgb.LGBMRegressor(learning_rate=0.01, metric='l2')
model.fit(X,y,eval_metric='l2')
filename = 'model/{}.pickle'.format('Uniform')
pickle.dump(model, open(filename,'wb'))

100%|██████████| 300/300 [14:45<00:00,  2.95s/it]


In [8]:
codes = ','.join(zz500['con_code'].values)
codes
data = ts_api.daily(ts_code=codes, start_date=str(date.today()-timedelta(7)).replace('-',''), end_date=str(date.today()).replace('-',''))
data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000009.SZ,20230303,12.58,12.66,12.52,12.52,12.58,-0.06,-0.4769,177434.39,223051.112
1,000012.SZ,20230303,6.96,6.97,6.89,6.95,6.93,0.02,0.2886,168550.22,116918.527
2,000021.SZ,20230303,13.01,13.26,12.81,12.99,12.89,0.10,0.7758,266189.12,346025.813
3,000027.SZ,20230303,6.28,6.33,6.26,6.32,6.27,0.05,0.7974,153671.95,96842.824
4,000031.SZ,20230303,3.76,3.80,3.76,3.79,3.78,0.01,0.2646,122207.13,46242.578
...,...,...,...,...,...,...,...,...,...,...,...
2995,688772.SH,20230224,21.30,21.39,20.70,20.80,21.18,-0.38,-1.7941,59493.44,124252.172
2996,688777.SH,20230224,91.05,96.30,91.05,95.16,88.19,6.97,7.9034,95594.32,900566.747
2997,688779.SH,20230224,15.79,15.80,15.56,15.68,15.74,-0.06,-0.3812,49388.19,77393.157
2998,688819.SH,20230224,37.01,37.03,36.51,36.76,36.86,-0.10,-0.2713,11675.03,42898.915


In [11]:
p = {}
for code in tqdm(zz500['con_code']):
    day_data = data[data['ts_code']==code]
    open_pct = (100*day_data['open'].loc[::-1].diff().values[1:]/day_data['open'].loc[::-1].values[:-1]).tolist()
    high_pct = (100*day_data['high'].loc[::-1].diff().values[1:]/day_data['high'].loc[::-1].values[:-1]).tolist()
    low_pct = (100*day_data['low'].loc[::-1].diff().values[1:]/day_data['low'].loc[::-1].values[:-1]).tolist()
    close_pct = (100*day_data['close'].loc[::-1].diff().values[1:]/day_data['close'].loc[::-1].values[:-1]).tolist()
    vol_pct = (100*day_data['vol'].loc[::-1].diff().values[1:]/day_data['vol'].loc[::-1].values[:-1]).tolist()
    amount_pct = (100*day_data['amount'].loc[::-1].diff().values[1:]/day_data['amount'].loc[::-1].values[:-1]).tolist()
    X = [open_pct + high_pct + low_pct + close_pct + vol_pct + amount_pct]
    filename = 'zzmodel/{}.pickle'.format(code[:-3])
    if os.path.exists(filename):
        model = pickle.load(open(filename,'rb'))
        p[model.predict(X)[0]] = code

100%|██████████| 500/500 [00:02<00:00, 223.51it/s]


In [12]:
[p[key] for key in sorted(p, reverse=True)[:10]]

['002156.SZ',
 '300058.SZ',
 '300418.SZ',
 '002396.SZ',
 '603127.SH',
 '300604.SZ',
 '002212.SZ',
 '603688.SH',
 '300676.SZ',
 '603816.SH']